# Improved Benchmark Processing with NF1-dataset using CytoSnake's `cp_process_singlecells` Workflow

This notebook uses the benchmark data obtained through the application of CytoSnake's `cp_process_singlecells` workflow. The collected data is then compiled into a unified benchmark profile, allowing comparisons with other generated benchmark profiles in this repo.

This benchmark profile was generated  by using the [`NF1-data`](https://github.com/WayScience/nf1_cellpainting_data)

**NOTE**: This benchmark profile only profiles the single-cell pipeline found [here](https://github.com/WayScience/nf1_cellpainting_data/blob/main/3.processing_features/2.pycytominer_singlecell_pipelines.ipynb).

The bulk benchmarking profile will be found in the `all-benchmarks/nf1_bulk_cp-process-singlecells_benchmarks` directory

In [1]:
import sys
import json
import pathlib
import subprocess
from datetime import datetime

import pandas as pd

sys.path.append("../../../")
from src.benchmark_utils import get_benchmark_files

## Generating CytoSnake `cp_process_singlecells` Benchmark Profile with NF1 Data

In this section, we are generating a benchmark profile for CytoSnake's `cp_process_singlecells` applied to the NF1 dataset. For additional information on the NF1 dataset, detailed documentation is available in its repository and can also be found in the controls directory of this repository.

In [2]:
# parameters
CWD_PATH = pathlib.Path(".").resolve()
BENCHMARK_DIR = pathlib.Path("./benchmarks/").resolve(strict=True)

In [3]:
# loading in file size data:
with open("./file_size.json", mode="r") as stream:
    file_sizes = json.load(stream)

# this is in MB
print(file_sizes)

{'Plate_1': 4.727, 'Plate_3': 551.629, 'Plate_3_prime': 444.938, 'Plate_4': 222.945, 'Plate_2': 30.598}


In [4]:
# converting .bin files into .json files (~2min)
# skip this block if the json files are already in the benchmarks/ directory
for bin_path in get_benchmark_files(BENCHMARK_DIR, ext="bin"):
    json_out = BENCHMARK_DIR / f"{bin_path.stem}.json"

    # # executing memray to convert bin files into json files
    memray_stats = subprocess.run(
        [
            "memray",
            "stats",
            "--json",
            "--output",
            str(json_out),
            "--force",
            str(bin_path),
        ],
        capture_output=True,
        check=True,
    )

    # stdout message
    print(
        f"{bin_path.relative_to(CWD_PATH)} was successfully converted into {json_out.relative_to(CWD_PATH)}"
    )

benchmarks/Plate_3_nf1_feature_select_benchmarks.bin was successfully converted into benchmarks/Plate_3_nf1_feature_select_benchmarks.json
benchmarks/Plate_3_prime_nf1_analysis_normalize_benchmarks.bin was successfully converted into benchmarks/Plate_3_prime_nf1_analysis_normalize_benchmarks.json
benchmarks/Plate_2_nf1_feature_select_benchmarks.bin was successfully converted into benchmarks/Plate_2_nf1_feature_select_benchmarks.json
benchmarks/Plate_1_nf1_analysis_normalize_benchmarks.bin was successfully converted into benchmarks/Plate_1_nf1_analysis_normalize_benchmarks.json
benchmarks/Plate_1_nf1_feature_select_benchmarks.bin was successfully converted into benchmarks/Plate_1_nf1_feature_select_benchmarks.json
benchmarks/Plate_2_nf1_analysis_annotate_benchmarks.bin was successfully converted into benchmarks/Plate_2_nf1_analysis_annotate_benchmarks.json
benchmarks/Plate_3_prime_nf1_analysis_annotate_benchmarks.bin was successfully converted into benchmarks/Plate_3_prime_nf1_analysis_

In [5]:
# applying time format
tformat = "%Y-%m-%d %H:%M:%S.%f"

# using all json files to compile benchmark profile
raw_benchmark_data = []
for json_path in get_benchmark_files(BENCHMARK_DIR, ext="json"):
    # collecting data from just file name
    plate_name = json_path.stem.split("_nf1_")[0]
    process_name = json_path.stem.split("_nf1_")[1].split("_benchmarks")[0]

    try:
        file_size = file_sizes[plate_name]
    except KeyError:
        file_size = "0"

    # opening json file to extract benchmark information
    with open(json_path, encoding="utf-8", mode="r") as contents:
        benchmark_data = json.load(contents)

        # accessing to all metadata from benchmarks
        meta_data = benchmark_data["metadata"]
        selected_data = {
            "pid": meta_data["pid"],
            "process_name": process_name,
            "input_data_name": plate_name,
            "start_time": datetime.strptime(meta_data["start_time"], tformat),
            "end_time": datetime.strptime(meta_data["end_time"], tformat),
            "time_duration": (
                datetime.strptime(meta_data["end_time"], tformat)
                - datetime.strptime(meta_data["start_time"], tformat)
            ).total_seconds(),
            "total_allocations": int(meta_data["total_allocations"]),
            "peak_memory": round(int(meta_data["peak_memory"]) / 1024**2, 3),
            "file_size": file_sizes[plate_name],
        }
        # append to list
        raw_benchmark_data.append(selected_data)

In [6]:
# generating benchmark profile and saving it
benchmark_profile = pd.DataFrame(data=raw_benchmark_data)
benchmark_profile

,pid,process_name,input_data_name,start_time,end_time,time_duration,total_allocations,peak_memory,file_size
0,2055507,feature_select,Plate_3_prime,2023-12-28 15:44:13.100,2023-12-28 15:45:03.253,50.153,29706986,735.230,444.938
1,2056506,feature_select,Plate_3,2023-12-28 15:45:49.986,2023-12-28 15:46:51.841,61.855,38371485,930.416,551.629
2,2039328,feature_select,Plate_2,2023-12-28 15:39:28.029,2023-12-28 15:39:34.022,5.993,2941689,192.806,30.598
3,2028154,analysis_annotate,Plate_1,2023-12-28 15:39:05.208,2023-12-28 15:39:07.372,2.164,1877874,43.003,4.727
4,2054817,analysis_annotate,Plate_3,2023-12-28 15:43:31.476,2023-12-28 15:45:34.531,123.055,104768964,1526.626,551.629
5,2033315,analysis_annotate,Plate_2,2023-12-28 15:39:13.335,2023-12-28 15:39:23.156,9.821,7699972,104.341,30.598
6,2051694,feature_select,Plate_4,2023-12-28 15:41:18.437,2023-12-28 15:41:53.149,34.712,17288206,484.301,222.945
7,2047513,analysis_normalize,Plate_4,2023-12-28 15:40:12.325,2023-12-28 15:41:16.722,64.397,43003649,1031.830,222.945
8,2047510,analysis_annotate,Plate_4,2023-12-28 15:40:12.214,2023-12-28 15:41:09.901,57.687,42705062,899.557,222.945
9,2034298,analysis_normalize,Plate_2,2023-12-28 15:39:15.598,2023-12-28 15:39:26.705,11.107,7816033,177.575,30.598


In [7]:
benchmark_profile.to_csv("nf1_cp_processing_singlecells_benchmark_profile.csv")